# PyCUDA: Python interface to CUDA

PyCUDA provides an interface from Python to *all* of CUDA. In addition, PyCUDA offers the following niceties:

1. **Memory management**: PyCUDA automatically cleans up objects at the end of their lifetimes, allowing you to write leak-free code effortlessly
2. **Error checking**: no need to manually check for errors; PyCUDA automatically translates CUDA errors into Python exceptions
3. **NumPy interaction:** PyCUDA provides an `ndarray`-like `GPUArray` class, which supports elementwise operations and broadcasting. NumPy arrays can easily be converted to `GPUArray`s and vice-versa
4. **Metaprogramming**: PyCUDA allows you to write more flexible, efficient kernels with configurable block sizes, unrolled loops, and configurable data types
5. **Rapid prototyping**: Allows rapid development of algorithms/kernels in e.g., Jupyter Notebooks

## GPUArray

In [10]:
from pycuda import autoinit
import pycuda.driver as cuda
import pycuda.gpuarray as gpuarray
import pycuda.compiler as compiler
import numpy as np

In [3]:
a = np.random.rand(5)
a_gpu = gpuarray.to_gpu(a)
print(a)

[ 0.916251    0.90730104  0.12345119  0.08681051  0.78787607]


In [4]:
a_gpu *= 2 # executes on GPU!

In [5]:
print(a_gpu)

[ 1.83250199  1.81460208  0.24690239  0.17362103  1.57575214]


## Example: matrix multiplication

In [6]:
kernel_text = """
__global__ void mm(double *A, double *B, double *C)
{
  
  // Block index
  const uint bx = blockIdx.x;
  const uint by = blockIdx.y;

  // Thread index
  const uint tx = threadIdx.x;
  const uint ty = threadIdx.y;

  // Index of the first sub-matrix of A processed by the block
  const uint aBegin = %(N)s * %(BLOCK_SIZE)s * by;
  // Index of the last sub-matrix of A processed by the block
  const uint aEnd = aBegin + %(N)s - 1;
  // Step size used to iterate through the sub-matrices of A
  const uint aStep = %(BLOCK_SIZE)s;

  // Index of the first sub-matrix of B processed by the block
  const uint bBegin = %(BLOCK_SIZE)s * bx;
  // Step size used to iterate through the sub-matrices of B
  const uint bStep = %(BLOCK_SIZE)s * %(N)s;

  // The element of the block sub-matrix that is computed
  // by the thread
  double Csub = 0;
  // Loop over all the sub-matrices of A and B required to
  // compute the block sub-matrix
  for (int a = aBegin, b = bBegin;
       a <= aEnd;
       a += aStep, b += bStep) 
    {
      // Shared memory for the sub-matrix of A
      __shared__ double As[%(BLOCK_SIZE)s][%(BLOCK_SIZE)s];
      // Shared memory for the sub-matrix of B
      __shared__ double Bs[%(BLOCK_SIZE)s][%(BLOCK_SIZE)s];

      // Load the matrices from global memory to shared memory
      // each thread loads one element of each matrix
      As[ty][tx] = A[a + %(N)s * ty + tx];
      Bs[ty][tx] = B[b + %(N)s * ty + tx];
      // Synchronize to make sure the matrices are loaded
      __syncthreads();

      // Multiply the two matrices together;
      // each thread computes one element
      // of the block sub-matrix
      for (int k = 0; k < %(BLOCK_SIZE)s; ++k)
        Csub += As[ty][k] * Bs[k][tx];

      // Synchronize to make sure that the preceding
      // computation is done before loading two new
      // sub-matrices of A and B in the next iteration
      __syncthreads();
    }

  // Write the block sub-matrix to global memory;
  // each thread writes one element
  const uint c = %(N)s * %(BLOCK_SIZE)s * by + %(BLOCK_SIZE)s * bx;
  C[c + %(N)s * ty + tx] = Csub;
}
"""

In [7]:
kernel_text = kernel_text % {
    'N': 1024,
    'BLOCK_SIZE': 32
}

In [8]:
print(kernel_text)


__global__ void mm(double *A, double *B, double *C)
{
  
  // Block index
  const uint bx = blockIdx.x;
  const uint by = blockIdx.y;

  // Thread index
  const uint tx = threadIdx.x;
  const uint ty = threadIdx.y;

  // Index of the first sub-matrix of A processed by the block
  const uint aBegin = 1024 * 32 * by;
  // Index of the last sub-matrix of A processed by the block
  const uint aEnd = aBegin + 1024 - 1;
  // Step size used to iterate through the sub-matrices of A
  const uint aStep = 32;

  // Index of the first sub-matrix of B processed by the block
  const uint bBegin = 32 * bx;
  // Step size used to iterate through the sub-matrices of B
  const uint bStep = 32 * 1024;

  // The element of the block sub-matrix that is computed
  // by the thread
  double Csub = 0;
  // Loop over all the sub-matrices of A and B required to
  // compute the block sub-matrix
  for (int a = aBegin, b = bBegin;
       a <= aEnd;
       a += aStep, b += bStep) 
    {
      // Shared memory for

In [11]:
mod = compiler.SourceModule(kernel_text)
func = mod.get_function('mm')
mm_gpu = func.prepare([np.intp, np.intp, np.intp])
# mm_gpu = func.prepare('PPP') # also works!

In [12]:
a = np.random.rand(1024, 1024)
b = np.random.rand(1024, 1024)
c = np.zeros([1024, 1024], dtype=np.float64)

a_d = gpuarray.to_gpu(a)
b_d = gpuarray.to_gpu(b)
c_d = gpuarray.to_gpu(c)

In [13]:
mm_gpu.prepared_call((32, 32, 1), (32, 32, 1), a_d.gpudata, b_d.gpudata, c_d.gpudata)

In [14]:
print(c_d)

[[ 246.85754872  243.95837449  240.59993294 ...,  250.27893264
   250.76185246  236.3539642 ]
 [ 255.2624859   251.66495653  244.46061962 ...,  245.64956071
   252.94923314  238.55513108]
 [ 265.22106953  265.23279476  260.72509666 ...,  264.66311797
   271.60582956  259.9790046 ]
 ..., 
 [ 250.64846012  239.2127767   243.99730386 ...,  240.59742457
   251.96533716  241.66013773]
 [ 264.47017362  255.2815124   252.71112128 ...,  254.16281217  263.9581633
   250.13513162]
 [ 263.97421922  258.632572    251.96585592 ...,  258.2554      262.68873799
   249.19610463]]


In [15]:
print(a.dot(b))

[[ 246.85754872  243.95837449  240.59993294 ...,  250.27893264
   250.76185246  236.3539642 ]
 [ 255.2624859   251.66495653  244.46061962 ...,  245.64956071
   252.94923314  238.55513108]
 [ 265.22106953  265.23279476  260.72509666 ...,  264.66311797
   271.60582956  259.9790046 ]
 ..., 
 [ 250.64846012  239.2127767   243.99730386 ...,  240.59742457
   251.96533716  241.66013773]
 [ 264.47017362  255.2815124   252.71112128 ...,  254.16281217  263.9581633
   250.13513162]
 [ 263.97421922  258.632572    251.96585592 ...,  258.2554      262.68873799
   249.19610463]]


In [20]:
start = cuda.Event()
end = cuda.Event()

start.record()
for i in range(100):
    mm_gpu.prepared_call((32, 32, 1), (32, 32, 1), a_d.gpudata, b_d.gpudata, c_d.gpudata)
end.record()
end.synchronize()

print("Average time for matrix multiply on GPU: {} seconds".format(start.time_till(end)*1e-3 / 100))

Average time for matrix multiply on GPU: 0.011200428466796874 seconds


In [21]:
start.record()
for i in range(100):
    a.dot(b)
end.record()
end.synchronize()

print("Average time for matrix multiply on CPU: {} seconds".format(start.time_till(end)*1e-3 / 100))

Average time for matrix multiply on CPU: 0.049225673828125 seconds


## Errors

In [22]:
mm_gpu.prepared_call((32, 32, 1), (33, 33, 1), a_d.gpudata, b_d.gpudata, c_d.gpudata) # wrong block size

LogicError: cuFuncSetBlockShape failed: invalid argument